In [ ]:
!which python

In [ ]:
import torch
import numpy as np
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import train_test_split
import re
import pandas as pd

In [ ]:
!which python

In [ ]:



currency_symbol_dict = {
    # --- SAFE: DISTINCT SYMBOLS ---
    'EUR': '€',       # Euro
    'GBP': '£',       # British Pound Sterling
    'INR': '₹',       # Indian Rupee
    'JPY': 'JP¥',     # Japanese Yen
    'CNY': 'CN¥',     # Chinese Yuan
    'RUB': '₽',       # Russian Ruble
    'NGN': '₦',       # Nigerian Naira
    'THB': '฿',       # Thai Baht
    'VND': '₫',       # Vietnamese Dong
    'KRW': '₩',       # South Korean Won
    'KPW': '₩',       # North Korean Won
    'TRY': '₺',       # Turkish Lira
    'GHS': '₵',       # Ghanaian Cedi
    'PHP': '₱',       # Philippine Peso
    'ILS': '₪',       # Israeli New Shekel
    'CRC': '₡',       # Costa Rican Colón
    'PLN': 'zł',      # Polish Zloty
    'CZK': 'Kč',      # Czech Koruna
    'MNT': '₮',       # Mongolian Tögrög
    'UAH': '₴',       # Ukrainian Hryvnia
    'GEL': '₾',       # Georgian Lari
    'AMD': '֏',       # Armenian Dram
    'AZN': '₼',       # Azerbaijani Manat
    'KZT': '₸',       # Kazakhstani Tenge
    'LAK': '₭',       # Lao Kip
    'KHR': '៛',       # Cambodian Riel
    'BGN': 'лв',      # Bulgarian Lev
    'MKD': 'ден',     # Macedonian Denar
    'RSD': 'дин',     # Serbian Dinar

    # --- ARABIC / DISTINCT SCRIPTS ---
    'AED': 'د.إ',     # UAE Dirham
    'SAR': 'ر.س',     # Saudi Riyal
    'DZD': 'د.ج',     # Algerian Dinar
    'LYD': 'ل.د',     # Libyan Dinar
    'MAD': 'د.م.',    # Moroccan Dirham
    'IQD': 'ع.د',     # Iraqi Dinar
    'IRR': '﷼',       # Iranian Rial
    'YER': '﷼',       # Yemeni Rial
    'OMR': 'ر.ع.',    # Omani Rial
    'QAR': 'ر.ق',     # Qatari Riyal
    'JOD': 'د.ا',     # Jordanian Dinar
    'KWD': 'د.ك',     # Kuwaiti Dinar
    'TND': 'د.ت',     # Tunisian Dinar
    'SDG': 'ج.س.',    # Sudanese Pound
    'LBP': 'ل.ل',     # Lebanese Pound
    'AFN': '؋',       # Afghan Afghani
    'BHD': '.د.ب',    # Bahraini Dinar
    'NPR': 'रू',      # Nepalese Rupee
    'BDT': '৳',       # Bangladeshi Taka

    # --- DOLLAR VARIANTS (KEPT AS REQUESTED) ---
    'USD': 'US$',       # US Dollar
    # 'ARS': '$',       # Argentine Peso
    # 'CLP': '$',       # Chilean Peso
    # 'COP': '$',       # Colombian Peso
    'MXN': 'MEX$',     # Mexican Peso
    'AUD': 'A$',      # Australian Dollar
    'CAD': 'C$',      # Canadian Dollar
    'SGD': 'S$',      # Singapore Dollar
    'BRL': 'R$',      # Brazilian Real
    'HKD': 'HK$',     # Hong Kong Dollar
    'NZD': 'NZ$',     # New Zealand Dollar
    'BSD': 'B$',      # Bahamian Dollar
    'BZD': 'BZ$',     # Belize Dollar
    'JMD': 'J$',      # Jamaican Dollar
    'TTD': 'TT$',     # Trinidad and Tobago Dollar
    'XCD': 'EC$',     # East Caribbean Dollar
    'FJD': 'FJ$',     # Fijian Dollar
    'SBD': 'SI$',     # Solomon Islands Dollar
    'KYD': 'CI$',     # Cayman Islands Dollar
    'LRD': 'L$',      # Liberian Dollar
    'NAD': 'N$',      # Namibian Dollar
    'UYU': '$U',      # Uruguayan Peso,
     'SRD': 'Sr$',     # Surinamese Dollar (Suggesting 'Sr$' for distinctness),
    'EGP':'E£',
    'BND': 'B$',
    'DOP':'RD$',
    'GYD':'G$',
    'KGS':'⃀',
    'MOP': 'MOP$',    # Macanese Pataca
    'PAB':'B/.',
    'PEN': 'S/.',      # Peruvian Sol
    'PYG': '₲',       # Paraguayan Guaraní,
    'SOS':'Sh.So.',
    'SYP': '£S',      # Syrian Pound
    'TOP': 'T$',      # Tongan Paʻanga
     'TWD': 'NT$',     # New Taiwan Dollar

}



currency_name_dict = {
    'USD': 'United States Dollar',
    'INR': 'Indian Rupee',
    'EUR': 'Euro',
    'GBP': 'British Pound Sterling',
    'JPY': 'Japanese Yen',
    'CHF': 'Swiss Franc',
    'KPW': 'North Korean Won',
    'PKR': 'Pakistani Rupee',
    'CNY': 'Chinese Yuan',
    'AUD': 'Australian Dollar',
    'CAD': 'Canadian Dollar',
    'MXN': 'Mexican Peso',
    'RUB': 'Russian Ruble',
    'ZAR': 'South African Rand',
    'NGN': 'Nigerian Naira',
    'THB': 'Thai Baht',
    'VND': 'Vietamese Dong',
    'IDR': 'Indonesian Rupiah',
    'KRW': 'South Korean Won',
    'PLN': 'Polish Złoty',
    'SEK': 'Swedish Krona',
    'TRY': 'Turkish Lira',
    'BRL': 'Brazilian Real',
    'AED': 'United Arab Emirates Dirham',
    'SAR': 'Saudi Riyal',
    'SCR': 'Seychellois Rupee',
    'YER': 'Yemeni Rial',
    'SLL': 'Sierra Leonean Leone',
    'MWK': 'Malawian Kwacha',
    'GHS': 'Ghanaian Cedi',
    'UGX': 'Ugandan Shilling',
    'TZS': 'Tanzanian Shilling',
    'XOF': 'West African CFA Franc',
    'DZD': 'Algerian Dinar',
    'EGP': 'Egyptian Pound',
    'PHP': 'Philippine Peso',
    'MYR': 'Malaysian Ringgit',
    'LKR': 'Sri Lankan Rupee',
    'NPR': 'Nepalese Rupee',
    'BDT': 'Bangladeshi Taka',
    'OMR': 'Omani Rial',
    'SGD': 'Singapore Dollar',

    'AFN': 'Afghan Afghani',
    'ALL': 'Albanian Lek',
    'AMD': 'Armenian Dram',
    'AOA': 'Angolan Kwanza',
    'ARS': 'Argentine Peso',
    'AZN': 'Azerbaijani Manat',
    'BAM': 'Bosnia and Herzegovina Convertible Mark',
    'BBD': 'Barbadian Dollar',
    'BGN': 'Bulgarian Lev',
    'BHD': 'Bahraini Dinar',
    'BIF': 'Burundian Franc',
    'BMD': 'Bermudian Dollar',
    'BND': 'Brunei Dollar',
    'BOB': 'Bolivian Boliviano',
    'BSD': 'Bahamian Dollar',
    'BTN': 'Bhutanese Ngultrum',
    'BWP': 'Botswana Pula',
    'BYN': 'Belarusian Ruble',
    'BZD': 'Belize Dollar',
    'CDF': 'Congolese Franc',
    'CLP': 'Chilean Peso',
    'COP': 'Colombian Peso',
    'CRC': 'Costa Rican Colón',
    'CUP': 'Cuban Peso',
    'CVE': 'Cape Verdean Escudo',
    'CZK': 'Czech Koruna',
    'DJF': 'Djiboutian Franc',
    'DKK': 'Danish Krone',
    'DOP': 'Dominican Peso',
    'ERN': 'Eritrean Nakfa',
    'ETB': 'Ethiopian Birr',
    'FJD': 'Fijian Dollar',
    'GEL': 'Georgian Lari',
    'GMD': 'Gambian Dalasi',
    'GNF': 'Guinean Franc',
    'GTQ': 'Guatemalan Quetzal',
    'GYD': 'Guyanese Dollar',
    'HKD': 'Hong Kong Dollar',
    'HNL': 'Honduran Lempira',
    'HTG': 'Haitian Gourde',
    'HUF': 'Hungarian Forint',
    'ILS': 'Israeli New Shekel',
    'IQD': 'Iraqi Dinar',
    'IRR': 'Iranian Rial',
    'ISK': 'Icelandic Króna',
    'JMD': 'Jamaican Dollar',
    'JOD': 'Jordanian Dinar',
    'KES': 'Kenyan Shilling',
    'KGS': 'Kyrgyzstani Som',
    'KHR': 'Cambodian Riel',
    'KMF': 'Comorian Franc',
    'KWD': 'Kuwaiti Dinar',
    'KYD': 'Cayman Islands Dollar',
    'KZT': 'Kazakhstani Tenge',
    'LAK': 'Lao Kip',
    'LBP': 'Lebanese Pound',
    'LRD': 'Liberian Dollar',
    'LSL': 'Lesotho Loti',
    'LYD': 'Libyan Dinar',
    'MAD': 'Moroccan Dirham',
    'MDL': 'Moldovan Leu',
    'MGA': 'Malagasy Ariary',
    'MKD': 'Macedonian Denar',
    'MMK': 'Myanmar Kyat',
    'MNT': 'Mongolian Tögrög',
    'MOP': 'Macanese Pataca',
    'MRU': 'Mauritanian Ouguiya',
    'MUR': 'Mauritian Rupee',
    'MVR': 'Maldivian Rufiyaa',
    'MZN': 'Mozambican Metical',
    'NAD': 'Namibian Dollar',
    'NIO': 'Nicaraguan Córdoba',
    'NOK': 'Norwegian Krone',
    'NZD': 'New Zealand Dollar',
    'PAB': 'Panamanian Balboa',
    'PEN': 'Peruvian Sol',
    'PGK': 'Papua New Guinean Kina',
    'PYG': 'Paraguayan Guaraní',
    'QAR': 'Qatari Riyal',
    'RON': 'Romanian Leu',
    'RSD': 'Serbian Dinar',
    'RWF': 'Rwandan Franc',
    'SBD': 'Solomon Islands Dollar',
    'SDG': 'Sudanese Pound',
    'SHP': 'Saint Helena Pound',
    'SOS': 'Somali Shilling',
    'SRD': 'Surinamese Dollar',
    'SSP': 'South Sudanese Pound',
    'STN': 'São Tomé and Príncipe Dobra',
    'SVC': 'Salvadoran Colón',
    'SYP': 'Syrian Pound',
    'SZL': 'Eswatini Lilangeni',
    'TJS': 'Tajikistani Somoni',
    'TMT': 'Turkmenistani Manat',
    'TND': 'Tunisian Dinar',
    'TOP': 'Tongan Paʻanga',
    'TTD': 'Trinidad and Tobago Dollar',
    'TWD': 'New Taiwan Dollar',
    'UAH': 'Ukrainian Hryvnia',
    'UYU': 'Uruguayan Peso',
    'UZS': 'Uzbekistani Som',
    'VES': 'Venezuelan Bolívar',
    'VUV': 'Vanuatu Vatu',
    'WST': 'Samoan Tālā',
    'XAF': 'Central African CFA Franc',
    'XCD': 'East Caribbean Dollar',
    'XPF': 'CFP Franc',
    'ZMW': 'Zambian Kwacha',
    'ZWG': 'Zimbabwean Gold'
}
    # 'USD': '$',       # US Dollar
    # 'INR': '₹',       # Indian Rupee
    # 'EUR': '€',       # Euro
    # 'GBP': '£',       # British Pound Sterling

currency_identifiers = ['EURO', 'EUR', 'USD', '$','€','£']
currency_pattern = '|'.join([re.escape(str(identifier)) for identifier in currency_identifiers])

In [ ]:
currency_name_dict = {k: " " + v + " " for k, v in currency_name_dict.items()}

In [ ]:
currency_name_dict

In [ ]:
len(currency_symbol_dict)

In [ ]:
len(currency_name_dict)

In [ ]:
import os
os.environ["HF_TOKEN"] = "your_token"

In [ ]:
# Load and filter FinanceMTEB/financial_phrasebank dataset
dataset_name = "FinanceMTEB/financial_phrasebank"
print(f"Loading dataset: {dataset_name}")
ds_fin = load_dataset(dataset_name)

filtered_ds_fin = {}
for split_name, dataset_split in ds_fin.items():
    if split_name == 'train':
        print(f"Applying currency pattern filter for split: {split_name}")
        filtered_split = dataset_split.filter(lambda example: bool(re.search(currency_pattern, example['text'])))
        filtered_ds_fin[split_name] = filtered_split
    else:
        print(f"Applying currency pattern filter for split: {split_name}")
        filtered_split = dataset_split.filter(lambda example: bool(re.search(currency_pattern, example['text'])))
        filtered_ds_fin[split_name] = filtered_split

train_texts = filtered_ds_fin['train']['text']
train_labels = filtered_ds_fin['train']['label']
test_texts = filtered_ds_fin['test']['text']
test_labels = filtered_ds_fin['test']['label']

In [ ]:
REPLACEMENT_TYPE='SYMBOL'
# REPLACEMENT_TYPE='ACRONYM'
# REPLACEMENT_TYPE='NAME'


unique_symbols = list(currency_symbol_dict.values())
unique_iso_codes = list(list(currency_symbol_dict.keys()))
target_replacement_strings = list(set(unique_symbols).union(set(unique_iso_codes)))[0:4]

target_replacement_strings = list(set(unique_symbols).union(set(unique_iso_codes)))[0:4]



if REPLACEMENT_TYPE=='SYMBOL':
    target_replacement_strings = list(set(unique_symbols))

if REPLACEMENT_TYPE=='NAME':
    target_replacement_strings = list(currency_name_dict.values())
    target_replacement_strings = [" " + v + " " for v in target_replacement_strings]


if REPLACEMENT_TYPE=='ACRONYM':
    target_replacement_strings = list(set(unique_iso_codes))


print(f"Total number of unique currency replacement strings: {len(target_replacement_strings)}")
print(f"First 10 unique currency replacement strings: {target_replacement_strings[:10]}")

In [ ]:
currency_pattern

In [ ]:
print(f"Current currency_pattern: '{currency_pattern}'")
# The pattern 'EURO' is listed before '\\ EUR', ensuring it is prioritized if both could match at a given position.

In [ ]:
# Compile regex outside the loop for efficiency
compiled_currency_pattern = re.compile(currency_pattern, flags=re.IGNORECASE)
compiled_currency_pattern
len(test_texts)

original_texts_for_df = []
modified_texts_for_df = []
processing_status_for_df = []
true_labels_for_df = []

for i, text in enumerate(test_texts):
    found_currencies = re.findall(compiled_currency_pattern, text)

    if found_currencies:
        unique_found_currencies = set(found_currencies)

        if len(unique_found_currencies) == 1:
            modified_text = compiled_currency_pattern.sub("****[REPLACE]****", text)
            processing_status = 'Processed (Single Currency)'
        else:
            modified_text = text
            processing_status = 'Skipped (Multiple Currencies)'
    else:
        modified_text = text
        processing_status = 'No Currency Found'

    original_texts_for_df.append(text)
    modified_texts_for_df.append(modified_text)
    processing_status_for_df.append(processing_status)
    true_labels_for_df.append(test_labels[i]) # Append the corresponding true label

before_after_df = pd.DataFrame({
    'original_text': original_texts_for_df,
    'modified_text': modified_texts_for_df,
    'processing_status': processing_status_for_df,
    'true_label': true_labels_for_df # Add the true_label column
})

print("DataFrame with original, modified texts, processing status, and true labels created successfully.")
display(before_after_df.head())

In [ ]:
len(before_after_df)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
len(before_after_df)

In [ ]:
display(before_after_df.sort_values(by='processing_status'))

In [ ]:
currency_pattern

In [ ]:
before_after_df = before_after_df[before_after_df['processing_status'] != 'Skipped (Multiple Currencies)']

print("DataFrame filtered successfully to exclude 'Skipped (Multiple Currencies)' rows.")
display(before_after_df)

In [ ]:
test_texts = before_after_df['original_text'].tolist()
test_labels = before_after_df['true_label'].tolist()

In [ ]:
len(before_after_df)

<!-- remove_subset_string = "lemiste City is the environment" -->


In [ ]:
remove_subset_string = "lemiste City is the environment"


In [ ]:
len(before_after_df)

In [ ]:

before_after_df = before_after_df[
    ~before_after_df['original_text'].str.contains(remove_subset_string, case=False, na=False, regex=False)
]

In [ ]:
len(before_after_df)

In [ ]:
display(before_after_df.sort_values(by='processing_status'))

In [ ]:
before_after_df=before_after_df.head(3)

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score
import pandas as pd

def calculate_metrics(group):
    if len(group) == 0:
        return pd.Series({'accuracy': 0.0, 'f1_score': 0.0, 'macro_precision': 0.0})

    accuracy = accuracy_score(group['numerical_true_label'], group['numerical_modified_prediction'])
    f1 = f1_score(group['numerical_true_label'], group['numerical_modified_prediction'], average='weighted', zero_division=0)
    macro_precision = precision_score(group['numerical_true_label'], group['numerical_modified_prediction'], average='macro', zero_division=0)
    return pd.Series({'accuracy': accuracy, 'f1_score': f1, 'macro_precision': macro_precision})

print("Defined calculate_metrics function, now including macro_precision.")

In [ ]:

# target_models = ["ProsusAI/finbert","FinLang/finance-embeddings-investopedia"]#,"intfloat/multilingual-e5-large", "intfloat/multilingual-e5-large-instruct","Alibaba-NLP/gte-large-en-v1.5","Alibaba-NLP/gte-multilingual-base","jinaai/jina-embeddings-v3",\

#                               # "jinaai/jina-embeddings-v2-base-en","BAAI/bge-m3","Snowflake/snowflake-arctic-embed-m-v1.5","sentence-transformers/paraphrase-multilingual-mpnet-base-v2","sentence-transformers/all-mpnet-base-v2",\

#                               # "Qwen/Qwen3-Embedding-0.6B","Qwen/Qwen3-Embedding-4B","Alibaba-NLP/gte-Qwen2-1.5B-instruct","google/embeddinggemma-300M",]




target_models = [ "Qwen/Qwen3-Embedding-0.6B","google/embeddinggemma-300M","Qwen/Qwen3-Embedding-4B", "ProsusAI/finbert","FinLang/finance-embeddings-investopedia","intfloat/multilingual-e5-large", "intfloat/multilingual-e5-large-instruct","Alibaba-NLP/gte-large-en-v1.5","Alibaba-NLP/gte-multilingual-base",\

                              "jinaai/jina-embeddings-v2-base-en","BAAI/bge-m3","Snowflake/snowflake-arctic-embed-m-v1.5","sentence-transformers/paraphrase-multilingual-mpnet-base-v2","sentence-transformers/all-mpnet-base-v2",\

                             ]
#"","Alibaba-NLP/gte-Qwen2-1.5B-instruct",,]


In [ ]:
sorted(target_models)

In [ ]:
len(target_models)

In [ ]:
device='cuda'

In [ ]:
import warnings
from sklearn.exceptions import ConvergenceWarning
import numpy as np



import torch
from transformers import pipeline, AutoModelForSequenceClassification, AutoTokenizer
from transformers.pipelines import Pipeline
from transformers.pipelines import PIPELINE_REGISTRY


class FinBERTPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        preprocess_kwargs = {}
        if "max_length" in kwargs:
            preprocess_kwargs["max_length"] = kwargs["max_length"]
        return {}, preprocess_kwargs, {}

    def preprocess(self, inputs, **kwargs):
        tokenizer = self.tokenizer
        return tokenizer(inputs, return_tensors=self.framework, padding=True, truncation=True, **kwargs)

    def _forward(self, model_inputs, **kwargs):
        with torch.no_grad():
            outputs = self.model(**model_inputs, output_hidden_states=True)
            # Typically, the last hidden state of the [CLS] token is used as the sentence embedding
            cls_token_embeddings = outputs.hidden_states[-1][:, 0, :].cpu().numpy()
        return {"embeddings": cls_token_embeddings}

    def postprocess(self, model_outputs, **kwargs):
        return model_outputs["embeddings"]

PIPELINE_REGISTRY.register_pipeline(
    "finbert-embedding",
    pipeline_class=FinBERTPipeline,
    pt_model=AutoModelForSequenceClassification,
    default={"model": "ProsusAI/finbert", "revision": "main"}
)

print("FinBERTPipeline class defined and registered.")


def embed_using_finbert(pipe, list_text):

    all_embeddings_list = pipe(list_text, padding=True, truncation=True, max_length=128)

    return all_embeddings_list



finbert_model = AutoModelForSequenceClassification.from_pretrained('ProsusAI/finbert').to(device)
finbert_tokenizer = AutoTokenizer.from_pretrained('ProsusAI/finbert')
finbert_pipe = pipeline(
    "finbert-embedding",
    model=finbert_model,
    tokenizer=finbert_tokenizer,
    device=0 if device == "cuda" else -1 # Use GPU 0 if CUDA, else CPU -1
)
currency_embeddings_map = embed_using_finbert(finbert_pipe, ['a','b'])





def analyze_currency_impact_exhaustive(logistic_predictor, model_name, embedding_model, test_texts, test_labels, test_embeddings, currency_pattern, single_replacement_string):
    impactful_changes = []
    modified_texts_to_encode = []
    original_prediction_list = []
    original_proba_list = []
    true_label_list = []
    original_text_list = []


    compiled_currency_pattern = re.compile(currency_pattern, flags=re.IGNORECASE)

    for i, text in enumerate(test_texts):
        # Get probabilities for the original text
        original_probas = logistic_predictor.predict_proba(test_embeddings[i].reshape(1, -1))[0]
        original_prediction = np.argmax(original_probas) # Derive class label from probabilities

        true_label = test_labels[i]

        if compiled_currency_pattern.search(text):
            # Replace all occurrences of currency mentions with the single_replacement_string
            modified_text = compiled_currency_pattern.sub(single_replacement_string, text)
            modified_texts_to_encode.append(modified_text)
            original_prediction_list.append(original_prediction)
            original_proba_list.append(original_probas)
            true_label_list.append(true_label)
            original_text_list.append(text)
        else:
            continue

    if modified_texts_to_encode:
        modified_embeddings = encode_text(modified_texts_to_encode[:], model_name, embedding_model).squeeze()
        modified_probas = logistic_predictor.predict_proba(modified_embeddings)
        modified_predictions = np.argmax(modified_probas, axis=1) # Derive class labels from probabilities

        for j in range(len(modified_texts_to_encode)):
            impactful_changes.append({
                'original_text': original_text_list[j],
                'replaced_with': single_replacement_string,
                'modified_text': modified_texts_to_encode[j],
                'original_prediction': original_prediction_list[j],
                'original_probabilities': original_proba_list[j].tolist(),
                'modified_prediction': modified_predictions[j],
                'modified_probabilities': modified_probas[j].tolist(),
                'true_label': true_label_list[j]
            })

    print('len(impactful_changes)', len(impactful_changes))
    return impactful_changes

print("Re-defined analyze_currency_impact_exhaustive function to include probabilities.")

In [ ]:
from transformers import AutoModel
# Initialize the model
model = AutoModel.from_pretrained("jinaai/jina-embeddings-v4", trust_remote_code=True)
model.to("cuda")

In [ ]:
model = SentenceTransformer("jinaai/jina-embeddings-v3", trust_remote_code=True)

In [ ]:
# !pip install transformers>=4.52.0 torch>=2.6.0 peft>=0.15.2 torchvision pillow
from transformers import AutoModel
# Initialize the model
model = AutoModel.from_pretrained("jinaai/jina-embeddings-v4", trust_remote_code=True)
model.to("cuda")

In [ ]:
def encode_text(list_text, model_name, embedding_model):
    if model_name == 'ProsusAI/finbert':
        return np.array(embed_using_finbert(finbert_pipe, list_text))
    else:
        return embedding_model.encode(list_text, convert_to_numpy=True)

In [ ]:
replacement_types_to_process = ['SYMBOL', 'ACRONYM', 'NAME']


In [ ]:
import os
import warnings
import numpy as np
import pandas as pd
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.exceptions import ConvergenceWarning
import re


currency_identifiers = ['EURO', 'EUR', 'USD', '$','€','£']
currency_pattern = '|'.join([re.escape(str(identifier)) for identifier in currency_identifiers])

# Load and filter the FinanceMTEB/financial_phrasebank dataset ---
dataset_name = "FinanceMTEB/financial_phrasebank"
print(f"Loading dataset: {dataset_name}")
ds_fin = load_dataset(dataset_name)

filtered_ds_fin = {}
for split_name, dataset_split in ds_fin.items():
    if split_name == 'train':
        print(f"Applying currency pattern filter for split: {split_name}")
        filtered_split = dataset_split.filter(lambda example: bool(re.search(currency_pattern, example['text'])))
        filtered_ds_fin[split_name] = filtered_split
    else:
        print(f"Applying currency pattern filter for split: {split_name}")
        filtered_split = dataset_split.filter(lambda example: bool(re.search(currency_pattern, example['text'])))
        filtered_ds_fin[split_name] = filtered_split

original_train_texts = filtered_ds_fin['train']['text']
original_train_labels = filtered_ds_fin['train']['label']
original_test_texts = filtered_ds_fin['test']['text']
original_test_labels = filtered_ds_fin['test']['label']

compiled_currency_pattern = re.compile(currency_pattern, flags=re.IGNORECASE)

new_test_texts = []
new_test_labels = []


substring_to_remove = "lemiste City is the environment"
for i, text in enumerate(original_test_texts):

    if substring_to_remove in text:
        continue

    found_currencies = re.findall(compiled_currency_pattern, text)
    if found_currencies:
        unique_found_currencies = set(found_currencies)
        if len(unique_found_currencies) == 1:
            new_test_texts.append(text)
            new_test_labels.append(original_test_labels[i])
test_texts = new_test_texts[0:10]
test_labels = new_test_labels[0:10]
train_texts = original_train_texts
train_labels = original_train_labels

print(f"Number of test samples after filtering for single currency mentions: {len(test_texts)}")

print('test_texts ')
print(test_texts)


# "ProsusAI/finbert"
print(sorted(target_models))


# "mixedbread-ai/mxbai-embed-large-v1"
# Define label_map for display
label_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}
sentiment_to_numerical = {'Negative': -1, 'Neutral': 0, 'Positive': 1}

output_dir = 'task_financial_phrasebank/
os.makedirs(output_dir, exist_ok=True)

warnings.filterwarnings("ignore", category=ConvergenceWarning)

for current_replacement_type in replacement_types_to_process:
    print(f"\n--- Processing Replacement Type: {current_replacement_type} ---")

    current_target_replacement_strings = []
    if current_replacement_type == 'SYMBOL':
        current_target_replacement_strings = list(set(currency_symbol_dict.values()))
    elif current_replacement_type == 'NAME':
        current_target_replacement_strings = list(currency_name_dict.values())
        current_target_replacement_strings = [" " + v + " " for v in current_target_replacement_strings]
    elif current_replacement_type == 'ACRONYM':
        current_target_replacement_strings = list(set(currency_symbol_dict.keys()))

    all_exhaustive_analysis_results = [] # Reset for each replacement type

    for model_name in target_models:
        print(f"--- Evaluating model: {model_name} for replacement type {current_replacement_type} ---")

        try:

            if model_name=='ProsusAI/finbert':
                embedding_model = None
                print('model_name', model_name)
            else:
              embedding_model = SentenceTransformer(model_name, device=device, trust_remote_code=True)
        except Exception as e:
            print(f"Error loading {model_name}: {e}. Skipping this model.")
            continue

        train_embeddings = np.array(encode_text(train_texts[:], model_name, embedding_model)).squeeze()
        test_embeddings = np.array(encode_text(test_texts[:], model_name, embedding_model)).squeeze()

        print('train_embeddings', train_embeddings.shape)

        logistic_model = LogisticRegression(max_iter=1000)
        logistic_model.fit(train_embeddings, train_labels)

        model_exhaustive_summary = {
            'model_name': model_name,
            'replacement_impact': {}
        }

        for replacement_str in current_target_replacement_strings:
            impact_changes = analyze_currency_impact_exhaustive(
                logistic_model, model_name, embedding_model, test_texts, test_labels, test_embeddings, currency_pattern, replacement_str
            )
            model_exhaustive_summary['replacement_impact'][replacement_str] = impact_changes

        all_exhaustive_analysis_results.append(model_exhaustive_summary)

    for model_result in all_exhaustive_analysis_results:
        model_name_for_save = model_result['model_name']
        model_specific_impact_data = []
        for replacement_str, impact_list in model_result['replacement_impact'].items():
            for item in impact_list:
                item_copy = item.copy()
                item_copy['model_name'] = model_name_for_save
                item_copy['replacement_string'] = replacement_str
                model_specific_impact_data.append(item_copy)

        if model_specific_impact_data:
            df_model_specific = pd.DataFrame(model_specific_impact_data)
            clean_model_name = model_name_for_save.replace('/', '_').replace('-', '_')
            file_name = f'comprehensive_impact_df_task_FinPhrase_{current_replacement_type}_{clean_model_name}.pkl'
            file_path_save = os.path.join(output_dir, file_name)
            df_model_specific.to_pickle(file_path_save)
            print(f"DataFrame for model '{model_name_for_save}' and replacement type '{current_replacement_type}' saved to '{file_path_save}'")




In [ ]:

target_models = [ "Qwen/Qwen3-Embedding-0.6B","google/embeddinggemma-300M", "ProsusAI/finbert","FinLang/finance-embeddings-investopedia","intfloat/multilingual-e5-large", "intfloat/multilingual-e5-large-instruct","Alibaba-NLP/gte-large-en-v1.5","Alibaba-NLP/gte-multilingual-base",\

                            "jinaai/jina-embeddings-v2-base-en","BAAI/bge-m3","Snowflake/snowflake-arctic-embed-m-v1.5","sentence-transformers/paraphrase-multilingual-mpnet-base-v2","sentence-transformers/all-mpnet-base-v2",\

                            ]
#"","Alibaba-NLP/gte-Qwen2-1.5B-instruct",,]

target_models+=["sentence-transformers/LaBSE"]




In [ ]:
sorted(target_models)

In [ ]:
len(target_models)

In [ ]:
# --- Step 1: Regenerate and save comprehensive_impact_df for each replacement type ---
replacement_types_to_process = ['SYMBOL', 'ACRONYM', 'NAME']


In [ ]:
# output_dir

In [ ]:

# Load and combine all comprehensive_impact_df files
all_comprehensive_impact_dfs = []
for replacement_type_load in replacement_types_to_process:
    for model_name_load in target_models: # Loop through models again to load model-specific files
        clean_model_name = model_name_load.replace('/', '_').replace('-', '_')
        file_path_load = os.path.join(output_dir, f'comprehensive_impact_df_task_FinPhrase_{replacement_type_load}_{clean_model_name}.pkl')
        try:
            df_temp = pd.read_pickle(file_path_load)
            df_temp['replacement_type'] = replacement_type_load
            # 'model_name' column should already be present in df_temp from saving
            all_comprehensive_impact_dfs.append(df_temp)
            print(f"Loaded {replacement_type_load} data for model {model_name_load}.")
        except FileNotFoundError:
            print(f"No data found for model {model_name_load} and replacement type {replacement_type_load}. Skipping.")

combined_impact_df = pd.concat(all_comprehensive_impact_dfs, ignore_index=True)
print("\nCombined DataFrame 'combined_impact_df' created.")

# Convert numerical labels to string names and vice versa
combined_impact_df['original_prediction'] = combined_impact_df['original_prediction'].map(label_map)
combined_impact_df['modified_prediction'] = combined_impact_df['modified_prediction'].map(label_map)
combined_impact_df['true_label'] = combined_impact_df['true_label'].map(label_map)

combined_impact_df['numerical_modified_prediction'] = combined_impact_df['modified_prediction'].map(sentiment_to_numerical)
combined_impact_df['numerical_true_label'] = combined_impact_df['true_label'].map(sentiment_to_numerical)
print("Labels converted to string and numerical formats in combined_impact_df.")

#  Calculate metrics_df including macro_precision
metrics_df = combined_impact_df.groupby(['model_name', 'replacement_string']).apply(calculate_metrics).reset_index()

print("\nMetrics DataFrame 'metrics_df' recreated successfully, including macro_precision.")
display(metrics_df.head())

In [ ]:
# all_exhaustive_analysis_results

In [ ]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_colwidth', None)

In [ ]:

output_dir = '/outputs/Classification_Exp/'

In [ ]:
import pandas as pd
import os
import pickle

all_comprehensive_impact_dfs = []
replacement_types_to_load = ['SYMBOL', 'ACRONYM', 'NAME']


for REPLACEMENT_TYPE in replacement_types_to_load:
    for model_name_load in target_models:
        clean_model_name = model_name_load.replace('/', '_').replace('-', '_')
        print('clean_model_name', clean_model_name)
        file_path = os.path.join(output_dir, f'comprehensive_impact_df_task_FinPhrase_{REPLACEMENT_TYPE}_{clean_model_name}.pkl')
        try:
            df_temp = pd.read_pickle(file_path)
            df_temp['replacement_type'] = REPLACEMENT_TYPE
            all_comprehensive_impact_dfs.append(df_temp)
            print(f"DataFrame 'comprehensive_impact_df_task_FinPhrase_{REPLACEMENT_TYPE}_{clean_model_name}.pkl' loaded successfully.")
        except FileNotFoundError:
            print(f"No data found for model {model_name_load} and replacement type {REPLACEMENT_TYPE}. Skipping.")
        except EOFError:
            print(f"EOFError: File '{file_path}' is empty or corrupted. Skipping.")
        except Exception as e:
            print(f"An unexpected error occurred while loading '{file_path}': {e}. Skipping.")

In [ ]:
import pandas as pd

# Define label_map here to ensure it's available
label_map = {0: 'Negative', 1: 'Neutral', 2: 'Positive'}


combined_impact_df = pd.concat(all_comprehensive_impact_dfs, ignore_index=True)

# Convert numerical labels to string names using label_map
combined_impact_df['original_prediction'] = combined_impact_df['original_prediction'].map(label_map)
combined_impact_df['modified_prediction'] = combined_impact_df['modified_prediction'].map(label_map)
combined_impact_df['true_label'] = combined_impact_df['true_label'].map(label_map)

print("Combined DataFrame 'combined_impact_df' created and labels converted successfully.")
display(combined_impact_df.head())

**Reasoning**:
The subtask requires mapping string sentiment labels to numerical values and adding these as new columns to `combined_impact_df`. I will first define the mapping dictionary and then apply it to the specified columns. Finally, I'll display the head of the DataFrame to show the changes.



In [ ]:
sentiment_to_numerical = {'Negative': -1, 'Neutral': 0, 'Positive': 1}

combined_impact_df['numerical_modified_prediction'] = combined_impact_df['modified_prediction'].map(sentiment_to_numerical)
combined_impact_df['numerical_true_label'] = combined_impact_df['true_label'].map(sentiment_to_numerical)

print("Numerical sentiment columns added to combined_impact_df.")
display(combined_impact_df.head())

In [ ]:
combined_impact_df.head(5)

In [ ]:
combined_impact_df['model_name'].unique()

In [ ]:
from scipy.stats import entropy
import numpy as np

def calculate_prediction_entropy(predictions_series):
    """
    Calculates the entropy of a series of predictions.
    Args:
        predictions_series (pd.Series): A series of categorical predictions (e.g., 'Positive', 'Neutral', 'Negative').
    Returns:
        float: The entropy of the prediction distribution.
    """
    # Convert predictions to a probability distribution
    prob_dist = predictions_series.value_counts(normalize=True)
    # Calculate entropy
    return entropy(prob_dist)

# Group by model_name and original_text, then apply the entropy calculation
entropy_per_text_per_model_df = combined_impact_df.groupby(['model_name', 'original_text'])['modified_prediction'].apply(calculate_prediction_entropy).reset_index()
entropy_per_text_per_model_df.rename(columns={'modified_prediction': 'entropy'}, inplace=True)

print("DataFrame 'entropy_per_text_per_model_df' created successfully.")
display(entropy_per_text_per_model_df.head())



# Calculate mean, median, standard deviation, min, max, and count of zero entropy for each model
model_entropy_summary = entropy_per_text_per_model_df.groupby('model_name')['entropy'].agg([
    'mean', 'median', 'std', 'min', 'max',
    ('q25', lambda x: x.quantile(0.25)),
    ('q75', lambda x: x.quantile(0.75)),
    ('zero_entropy_count', lambda x: (x == 0).sum()),
    ('total_samples', 'count'),
    ('non_zero_entropy_samples', lambda x: (x > 0).sum())
]).reset_index()
model_entropy_summary.rename(columns={'mean': 'mean_entropy', 'median': 'median_entropy', 'std': 'std_dev_entropy', 'min': 'min_entropy', 'max': 'max_entropy'}, inplace=True)

print("Mean, median, std dev, min, max, q25, q75, zero entropy count, total samples, and non-zero entropy samples calculated for each model:")
display(model_entropy_summary)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Get unique model names
model_names = entropy_per_text_per_model_df['model_name'].unique()

for model_name in model_names:
    model_entropy_data = entropy_per_text_per_model_df[entropy_per_text_per_model_df['model_name'] == model_name]

    plt.figure(figsize=(10, 6))
    sns.histplot(model_entropy_data['entropy'], bins=30, kde=True)
    plt.title(f'Distribution of Entropy for Modified Predictions - Model: {model_name}')
    plt.xlabel('Entropy')
    plt.ylabel('Frequency')
    plt.grid(True)
    plt.show()

print("Histograms of entropy distribution for each model generated successfully.")

In [ ]:
!pip install seaborn

In [ ]:
output_dir

In [ ]:
from scipy.stats import entropy
import numpy as np

def calculate_prediction_entropy(predictions_series):
    """
    Calculates the entropy of a series of predictions.
    Args:
        predictions_series (pd.Series): A series of categorical predictions (e.g., 'Positive', 'Neutral', 'Negative').
    Returns:
        float: The entropy of the prediction distribution.
    """
    # Convert predictions to a probability distribution
    prob_dist = predictions_series.value_counts(normalize=True)
    # Calculate entropy
    return entropy(prob_dist)

# Group by model_name and original_text, then apply the entropy calculation
entropy_per_text_per_model_df = combined_impact_df.groupby(['model_name', 'original_text'])['modified_prediction'].apply(calculate_prediction_entropy).reset_index()
entropy_per_text_per_model_df.rename(columns={'modified_prediction': 'entropy'}, inplace=True)

print("DataFrame 'entropy_per_text_per_model_df' created successfully.")
display(entropy_per_text_per_model_df.head())



# Calculate mean, median, standard deviation, min, max, and count of zero entropy for each model
model_entropy_summary = entropy_per_text_per_model_df.groupby('model_name')['entropy'].agg([
    'mean', 'median', 'std', 'min', 'max',
    ('q25', lambda x: x.quantile(0.25)),
    ('q75', lambda x: x.quantile(0.75)),
    ('zero_entropy_count', lambda x: (x == 0).sum()),
    ('total_samples', 'count'),
    ('non_zero_entropy_samples', lambda x: (x > 0).sum())
]).reset_index()
model_entropy_summary.rename(columns={'mean': 'mean_entropy', 'median': 'median_entropy', 'std': 'std_dev_entropy', 'min': 'min_entropy', 'max': 'max_entropy'}, inplace=True)

print("Mean, median, std dev, min, max, q25, q75, zero entropy count, total samples, and non-zero entropy samples calculated for each model:")
display(model_entropy_summary)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

df_plot = entropy_per_text_per_model_df.copy()
df_plot['model_name_clean'] = df_plot['model_name'].apply(lambda x: x.split('/')[-1])
order = df_plot.groupby('model_name_clean')['entropy'].median().sort_values().index

sns.set_style("whitegrid")
sns.set_context("poster")
plt.figure(figsize=(20, 14))

ax = sns.boxplot(
    x='entropy',
    y='model_name_clean',
    data=df_plot,
    order=order,
    palette='viridis',
    width=0.6,
    linewidth=3, # Thicker lines for the boxes
    showfliers=True,
    flierprops={"marker": "o", "markersize": 5, "alpha": 0.2, "markerfacecolor": "gray"}
)

sns.pointplot(
    x='entropy',
    y='model_name_clean',
    data=df_plot,
    order=order,
    join=False,
    color='black',
    markers='D',
    scale=1.2,
    errorbar='sd',
    capsize=0.15,
    errwidth=4 # Thick error bars
)

plt.suptitle('Distribution of Sentiment Prediction Entropy Across Models',
          fontsize=26, fontweight='black', y=0.95)

plt.xlabel('Prediction Entropy Score',
           fontsize=32, fontweight='bold', labelpad=25)

plt.ylabel('',
           fontsize=32, fontweight='bold', labelpad=0)

plt.yticks(fontsize=24, fontweight='bold')
plt.xticks(fontsize=24, fontweight='bold')
sns.despine(left=True)
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.tight_layout(rect=[0, 0.03, 1, 0.98])

plt.savefig(f"{output_dir}/model_entropy_sentiment_plot.pdf",
            format="pdf", dpi=300, bbox_inches='tight')

plt.show()



In [ ]:
s# entropy_per_text_per_model_df


## Calculate Prediction Counts per Model, Currency, and Sentiment



In [ ]:
prediction_counts_df = combined_impact_df.groupby(['model_name', 'replacement_string', 'modified_prediction']).size().reset_index(name='count')

print("DataFrame 'prediction_counts_df' created successfully.")
display(prediction_counts_df.head())

In [ ]:
positive_predictions_per_currency_per_model_df = prediction_counts_df[prediction_counts_df['modified_prediction'] == 'Positive'].copy()
negative_predictions_per_currency_per_model_df = prediction_counts_df[prediction_counts_df['modified_prediction'] == 'Negative'].copy()
neutral_predictions_per_currency_per_model_df = prediction_counts_df[prediction_counts_df['modified_prediction'] == 'Neutral'].copy()

print("Separated prediction counts into sentiment-specific DataFrames.")
display(positive_predictions_per_currency_per_model_df.head())

In [ ]:
k = 5 # Number of top/bottom currencies to display

# Get unique model names
model_names = prediction_counts_df['model_name'].unique()

for model_name in model_names:
    print(f"\n--- Model: {model_name} ---")

    # Positive Predictions
    model_df = positive_predictions_per_currency_per_model_df[positive_predictions_per_currency_per_model_df['model_name'] == model_name].copy()
    model_df = model_df.sort_values(by='count', ascending=False).rename(columns={'count': 'positive_prediction_count'})

    print(f"\nTop {k} Positive Predictions for {model_name}:")
    top_k = model_df.head(k).drop(columns=['model_name', 'modified_prediction'])
    display(top_k)

    print(f"Bottom {k} Positive Predictions for {model_name}:")
    bottom_k = model_df.tail(k).drop(columns=['model_name', 'modified_prediction'])
    display(bottom_k)

    # Negative Predictions
    model_df_negative = negative_predictions_per_currency_per_model_df[negative_predictions_per_currency_per_model_df['model_name'] == model_name].copy()
    model_df_negative = model_df_negative.sort_values(by='count', ascending=False).rename(columns={'count': 'negative_prediction_count'})

    print(f"\nTop {k} Negative Predictions for {model_name}:")
    top_k_negative = model_df_negative.head(k).drop(columns=['model_name', 'modified_prediction'])
    display(top_k_negative)

    print(f"Bottom {k} Negative Predictions for {model_name}:")
    bottom_k_negative = model_df_negative.tail(k).drop(columns=['model_name', 'modified_prediction'])
    display(bottom_k_negative)

    # Neutral Predictions
    model_df_neutral = neutral_predictions_per_currency_per_model_df[neutral_predictions_per_currency_per_model_df['model_name'] == model_name].copy()
    model_df_neutral = model_df_neutral.sort_values(by='count', ascending=False).rename(columns={'count': 'neutral_prediction_count'})

    print(f"\nTop {k} Neutral Predictions for {model_name}:")
    top_k_neutral = model_df_neutral.head(k).drop(columns=['model_name', 'modified_prediction'])
    display(top_k_neutral)

    print(f"Bottom {k} Neutral Predictions for {model_name}:")
    bottom_k_neutral = model_df_neutral.tail(k).drop(columns=['model_name', 'modified_prediction'])
    display(bottom_k_neutral)


In [ ]:
negative_model_stats = negative_predictions_per_currency_per_model_df.groupby('model_name')['count'].describe()
print("Descriptive statistics for negative prediction counts per model:")
display(negative_model_stats)



positive_model_stats = positive_predictions_per_currency_per_model_df.groupby('model_name')['count'].describe()
print("Descriptive statistics for positive_model_stats prediction counts per model:")
display(positive_model_stats)



neutral_model_stats = neutral_predictions_per_currency_per_model_df.groupby('model_name')['count'].describe()
print("Descriptive statistics for neutral_model_stats prediction counts per model:")
display(neutral_model_stats)

**Reasoning**:
The next step is to generate a box plot to visualize the distribution of `negative_prediction_count` for each `model_name` using the `negative_predictions_per_currency_per_model_df` DataFrame, as specified in the task.



In [ ]:
model_names

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os

# --- STYLE SETTINGS ---
sns.set_style("whitegrid")
# 'poster' is the largest context, making lines and labels naturally massive
sns.set_context("poster")

# output_dir = "sentiment_distributions"
output_individual_plots = output_dir + '/sentiment_plots_normal/'
os.makedirs(output_individual_plots, exist_ok=True)

model_names = prediction_counts_df['model_name'].unique()

for model_name in model_names:
    clean_name = model_name.split('/')[-1]
    model_data = prediction_counts_df[prediction_counts_df['model_name'] == model_name]

    # Increased figure height slightly to accommodate larger fonts
    fig, axes = plt.subplots(1, 3, figsize=(24, 10), sharey=True)

    # MASSIVE MAIN TITLE
    fig.suptitle(f'Sentiment Label Distribution Density for {clean_name}',
                 fontsize=32, fontweight='black', y=1.01)

    sentiments = [
        ('Positive', '#3498db', axes[0]),
        ('Neutral', '#2ecc71', axes[1]),
        ('Negative', '#e74c3c', axes[2])
    ]

    for label, col, ax in sentiments:
        sentiment_data = model_data[model_data['modified_prediction'] == label]

        if not sentiment_data.empty:
            sns.histplot(x='count', data=sentiment_data, kde=True, ax=ax,
                         color=col, alpha=0.7, edgecolor='white', linewidth=3)

            # Subplot titles (Positive, Neutral, Negative)
            ax.set_title(label.upper(), fontsize=22, fontweight='bold', pad=20)
        else:
            ax.set_title(f'{label.upper()} (NO DATA)', fontsize=22, fontweight='bold', color='gray')
            ax.text(0.5, 0.5, 'No Data', ha='center', va='center',
                    transform=ax.transAxes, fontweight='bold', fontsize=24)

        # Clear local labels to use the global ones
        ax.set_xlabel('')
        ax.set_ylabel('')

        # TICKS: Larger and Bolder numbers
        ax.tick_params(axis='both', labelsize=24, pad=5)
        plt.setp(ax.get_xticklabels(), fontweight='bold')
        plt.setp(ax.get_yticklabels(), fontweight='bold')

    # --- ADD MASSIVE GLOBAL LABELS (TIGHTENED GAP) ---

    # Common X-axis label
    fig.supxlabel('Label Agreement (How many times the model picked this label per text)',
                  fontsize=30, fontweight='bold', y=0.05)

    # Common Y-axis label
    fig.supylabel('Number of Unique Sentences',
                  fontsize=30, fontweight='bold', x=0.01)

    # Adjusting layout for large fonts
    # rect=[left, bottom, right, top]
    sns.despine()
    plt.tight_layout(rect=[0.05, 0.1, 1, 0.95])

    safe_name = clean_name.replace('.', '_').replace('-', '_')
    file_path = f"{output_individual_plots}/stability_{safe_name}.pdf"
    plt.tight_layout()

    plt.savefig(file_path, format="pdf", dpi=300, bbox_inches='tight')
    plt.show()

print(f"\n✅ All high-visibility plots saved to '{output_dir}'.")

In [ ]:
output_dir

## Calculate Overall Sentiment Proportions per Model



In [ ]:
aggregated_predictions_per_model = prediction_counts_df.groupby(['model_name', 'modified_prediction'])['count'].sum().reset_index()

pivoted_predictions_df = aggregated_predictions_per_model.pivot(index='model_name', columns='modified_prediction', values='count').fillna(0)
pivoted_predictions_df.columns.name = None

pivoted_predictions_df['Total Predictions'] = pivoted_predictions_df['Positive'] + pivoted_predictions_df['Neutral'] + pivoted_predictions_df['Negative']

overall_sentiment_proportions = pivoted_predictions_df.copy()
overall_sentiment_proportions['Positive_Proportion'] = overall_sentiment_proportions['Positive'] / overall_sentiment_proportions['Total Predictions']
overall_sentiment_proportions['Neutral_Proportion'] = overall_sentiment_proportions['Neutral'] / overall_sentiment_proportions['Total Predictions']
overall_sentiment_proportions['Negative_Proportion'] = overall_sentiment_proportions['Negative'] / overall_sentiment_proportions['Total Predictions']

print("DataFrame 'overall_sentiment_proportions' created successfully with sentiment proportions.")
display(overall_sentiment_proportions.head())

In [ ]:
grouped_predictions = prediction_counts_df.groupby(['model_name', 'replacement_string', 'modified_prediction'])['count'].sum().reset_index()

currency_predictions_pivot_df = grouped_predictions.pivot_table(
    index=['model_name', 'replacement_string'],
    columns='modified_prediction',
    values='count'
).fillna(0).reset_index()

currency_predictions_pivot_df['Total_Predictions'] = currency_predictions_pivot_df['Positive'] + \
                                            currency_predictions_pivot_df['Neutral'] + \
                                            currency_predictions_pivot_df['Negative']

currency_predictions_pivot_df['Positive_Proportion'] = currency_predictions_pivot_df['Positive'] / currency_predictions_pivot_df['Total_Predictions']
# currency_predictions_pivot_df['Neutral_Proportion'] = currency_predictions_pivot_df['Neutral'] / currency_predictions_pivot_df['Total_Predictions']
# currency_predictions_pivot_df['Negative_Proportion'] = currency_predictions_pivot_df['Negative'] / currency_predictions_pivot_df['Total_Predictions']

currency_sentiment_proportions = currency_predictions_pivot_df.copy()

print("DataFrame 'currency_sentiment_proportions' created successfully with currency-specific sentiment proportions.")
display(currency_sentiment_proportions.head())

In [ ]:
bias_df = pd.merge(currency_sentiment_proportions, overall_sentiment_proportions, on='model_name', suffixes=('_currency', '_overall'))

bias_df['Positive_Bias'] = bias_df['Positive_Proportion_currency'] - bias_df['Positive_Proportion_overall']
# bias_df['Neutral_Bias'] = bias_df['Neutral_Proportion_currency'] - bias_df['Neutral_Proportion_overall']
# bias_df['Negative_Bias'] = bias_df['Negative_Proportion_currency'] - bias_df['Negative_Proportion_overall']

# bias_df['Neutral_Bias'] = bias_df['Neutral_Proportion_currency'] - bias_df['Neutral_Proportion_overall']
# bias_df['Negative_Bias'] = bias_df['Negative_Proportion_currency'] - bias_df['Negative_Proportion_overall']



print("DataFrame 'bias_df' created successfully with bias scores.")
display(bias_df.head())

In [ ]:
k = 5 # Number of top/bottom currencies to display

model_names = bias_df['model_name'].unique()

for model_name in model_names:
    print(f"\n--- Identifying Most Biased Currencies for Model: {model_name} ---")
    model_bias_df = bias_df[bias_df['model_name'] == model_name]

    # Positive Bias
    print(f"\nTop {k} Positive Bias (most positively biased) for {model_name}:")
    top_positive_bias = model_bias_df.sort_values(by='Positive_Bias', ascending=False).head(k)
    display(top_positive_bias[['replacement_string', 'Positive_Bias', 'Positive_Proportion_currency', 'Positive_Proportion_overall']])

    print(f"Bottom {k} Positive Bias (most negatively biased towards positive) for {model_name}:")
    bottom_positive_bias = model_bias_df.sort_values(by='Positive_Bias', ascending=True).head(k)
    display(bottom_positive_bias[['replacement_string', 'Positive_Bias', 'Positive_Proportion_currency', 'Positive_Proportion_overall']])

    # # Neutral Bias
    # print(f"\nTop {k} Neutral Bias (most neutral biased) for {model_name}:")
    # top_neutral_bias = model_bias_df.sort_values(by='Neutral_Bias', ascending=False).head(k)
    # display(top_neutral_bias[['replacement_string', 'Neutral_Bias', 'Neutral_Proportion_currency', 'Neutral_Proportion_overall']])

    # print(f"Bottom {k} Neutral Bias (most negatively biased towards neutral) for {model_name}:")
    # bottom_neutral_bias = model_bias_df.sort_values(by='Neutral_Bias', ascending=True).head(k)
    # display(bottom_neutral_bias[['replacement_string', 'Neutral_Bias', 'Neutral_Proportion_currency', 'Neutral_Proportion_overall']])

    # # Negative Bias
    # print(f"\nTop {k} Negative Bias (most negatively biased) for {model_name}:")
    # top_negative_bias = model_bias_df.sort_values(by='Negative_Bias', ascending=False).head(k)
    # display(top_negative_bias[['replacement_string', 'Negative_Bias', 'Negative_Proportion_currency', 'Negative_Proportion_overall']])

    # print(f"Bottom {k} Negative Bias (most positively biased towards negative) for {model_name}:")
    # bottom_negative_bias = model_bias_df.sort_values(by='Negative_Bias', ascending=True).head(k)
    # display(bottom_negative_bias[['replacement_string', 'Negative_Bias', 'Negative_Proportion_currency', 'Negative_Proportion_overall']])


In [ ]:
bias_df_percentages = bias_df.copy()

bias_df_percentages['Positive_Bias_Percentage'] = bias_df_percentages['Positive_Bias'] * 100
# bias_df_percentages['Neutral_Bias_Percentage'] = bias_df_percentages['Neutral_Bias'] * 100
# bias_df_percentages['Negative_Bias_Percentage'] = bias_df_percentages['Negative_Bias'] * 100

print("DataFrame 'bias_df_percentages' created with bias scores in percentage points.")
display(bias_df_percentages[['model_name', 'replacement_string', 'Positive_Bias_Percentage']].head())#, 'Neutral_Bias_Percentage', 'Negative_Bias_Percentage']].head())
# display(bias_df_percentages[['model_name', 'replacement_string', 'Positive_Bias_Percentage', 'Neutral_Bias_Percentage', 'Negative_Bias_Percentage']].head())

In [ ]:

bias_df_percentages = bias_df.copy()

bias_df_percentages['Positive_Relative_Change'] = (bias_df_percentages['Positive_Bias'] / bias_df_percentages['Positive_Proportion_overall']) * 100
bias_df_percentages.replace([np.inf, -np.inf], np.nan, inplace=True)

print("Updated DataFrame 'bias_df_percentages' with neutral and negative bias, and relative percentage changes.")
display(bias_df_percentages[['model_name', 'replacement_string','Positive_Relative_Change']].head())

In [ ]:
bias_df_percentages.head(10)

In [ ]:
currency_bias_summary = bias_df_percentages.groupby('replacement_string').agg(
    mean_positive_relative_change=('Positive_Relative_Change', 'mean'),
    std_positive_relative_change=('Positive_Relative_Change', 'std')
).reset_index()

print("DataFrame 'currency_bias_summary' created successfully with mean and standard deviation of positive relative change.")
display(currency_bias_summary.head())

In [ ]:
print("\nTop 5 most positive-biased currencies (highest mean_positive_relative_change):")
top_5_positive_bias = currency_bias_summary.sort_values(by='mean_positive_relative_change', ascending=False).head(5)
display(top_5_positive_bias)

print("\nTop 5 most negatively-biased currencies (lowest mean_positive_relative_change):")
top_5_negative_bias = currency_bias_summary.sort_values(by='mean_positive_relative_change', ascending=True).head(5)
display(top_5_negative_bias)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Sort the currencies by mean_positive_relative_change for better visualization
currency_bias_summary_sorted = currency_bias_summary.sort_values(by='mean_positive_relative_change', ascending=True)

# Get the top 10 most negatively-biased (lowest mean_positive_relative_change)
bottom_10_biased = currency_bias_summary_sorted.head(10)

# Get the top 10 most positive-biased (highest mean_positive_relative_change)
top_10_biased = currency_bias_summary_sorted.tail(10)

# Combine and sort for plotting
plot_data = pd.concat([bottom_10_biased, top_10_biased]).sort_values(by='mean_positive_relative_change', ascending=True)

plt.figure(figsize=(16, 10))

# Create a colormap that maps values to a diverging palette (e.g., coolwarm)
# This helps visualize positive and negative biases effectively
norm = plt.Normalize(plot_data['mean_positive_relative_change'].min(),
                     plot_data['mean_positive_relative_change'].max())
cmap = plt.cm.coolwarm
colors = cmap(norm(plot_data['mean_positive_relative_change']))

plt.barh(
    plot_data['replacement_string'],
    plot_data['mean_positive_relative_change'],
    xerr=plot_data['std_positive_relative_change'], # Standard deviation as error bars
    color=colors, # Apply the diverging color map
    edgecolor='black' # Add edge color for better separation
)

plt.title('Top/Bottom 10 Most Biased Currencies (Mean Positive Relative Change with Std Dev)', fontsize=18, fontweight='bold')
plt.xlabel('Mean Positive Relative Change (%)', fontsize=14)
plt.ylabel('Currency Replacement String', fontsize=14)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
k = 15
top_k_currencies = currency_bias_summary.nlargest(k, 'mean_positive_relative_change')['replacement_string'].tolist()
bottom_k_currencies = currency_bias_summary.nsmallest(k, 'mean_positive_relative_change')['replacement_string'].tolist()

selected_currencies = list(set(top_k_currencies + bottom_k_currencies))

filtered_bias_df_percentages = bias_df_percentages[bias_df_percentages['replacement_string'].isin(selected_currencies)].copy()

# Get the order of replacement strings based on their mean_positive_relative_change for consistent plotting
sorted_currency_order = currency_bias_summary.set_index('replacement_string').loc[selected_currencies].sort_values(by='mean_positive_relative_change', ascending=False).index.tolist()

# Convert replacement_string to a categorical type with the sorted order
filtered_bias_df_percentages['replacement_string'] = pd.Categorical(filtered_bias_df_percentages['replacement_string'], categories=sorted_currency_order, ordered=True)

# Sort the DataFrame by the categorical order
filtered_bias_df_percentages = filtered_bias_df_percentages.sort_values(by='replacement_string').reset_index(drop=True)

print(f"Filtered `bias_df_percentages` to include {len(selected_currencies)} unique currency replacement strings. Shape: {filtered_bias_df_percentages.shape}")
display(filtered_bias_df_percentages.head(20))

In [ ]:
top_k_currencies, bottom_k_currencies

In [ ]:
# filtered_bias_df_percentages

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

plt.rcParams['font.family'] = 'sans-serif'

preferred_fonts = ['Noto Color Emoji', 'Noto Sans CJK JP', 'Noto Sans Display', 'DejaVu Sans', 'Arial Unicode MS', 'Arial']

found_font_name = None
for font_name in preferred_fonts:
    try:
        font_path = fm.findfont(fm.FontProperties(family=font_name), fallback_to_default=False)
        found_font_name = fm.FontProperties(fname=font_path).get_name()
        plt.rcParams['font.sans-serif'] = [found_font_name]
        plt.rcParams['axes.unicode_minus'] = False # Fix minus sign for Unicode fonts
        print(f"Using Unicode font: {found_font_name}")
        break
    except ValueError:
        pass
    except UserWarning:
        pass

if not found_font_name:
    print("Warning: Could not find a suitable Unicode font. Currency symbols might not display correctly, defaulting to system font.")
    plt.rcParams['font.sans-serif'] = ['DejaVu Sans', 'Arial', 'Liberation Sans'] # Fallback list
    plt.rcParams['axes.unicode_minus'] = False


sns.set_style("whitegrid")
sns.set_context("poster")

plt.figure(figsize=(18, 12))



ax = sns.boxplot(
    x='Positive_Relative_Change',
    y='replacement_string',
    data=filtered_bias_df_percentages,
    hue='replacement_string',
    legend=False,
    palette='viridis',
    width=0.6,
    linewidth=2.5,
    whis=0,
    showfliers=True,  # Keep outliers visible
    # REDUCED markersize here (changed from 7 to 3)
    flierprops={"marker": "o", "markersize": 3, "alpha": 0.4}
)

sns.pointplot(
    x='Positive_Relative_Change',
    y='replacement_string',
    data=filtered_bias_df_percentages,
    linestyle='none',
    color='black',
    markers='D',
    markersize=15,
    errorbar='sd',
    capsize=0.15,      # Keeps the SD caps
    err_kws={'linewidth': 3}
)

plt.axvline(x=0, color='crimson', linestyle='-', linewidth=4, zorder=1)
plt.axhline(y=k - 0.5, color='black', linestyle=':', linewidth=3, alpha=0.8)

plt.text(ax.get_xlim()[1], k - 0.7, f' Top {k} ', va='bottom', ha='right', fontweight='bold', fontsize=16)
plt.text(ax.get_xlim()[1], k - 0.3, f' Bottom {k} ', va='top', ha='right', fontweight='bold', fontsize=16)

# Apply high-visibility styling
plt.title(f'Distribution of Positive Relative Change for Top/Bottom {k} Biased Currencies',
          fontsize=30, fontweight='black', pad=30)
plt.xlabel('Positive Relative Change (%)', fontsize=24, fontweight='bold', labelpad=20)
plt.ylabel('Currency Replacement String', fontsize=24, fontweight='bold', labelpad=20)

plt.xticks(fontsize=18, fontweight='bold')
plt.yticks(fontsize=18, fontweight='bold')

plt.grid(axis='x', linestyle='--', alpha=0.6)
sns.despine(left=True)

plt.tight_layout()
plt.show()

In [ ]:
k = 10
print(f"Set k to: {k}")

In [ ]:
model_names = bias_df_percentages['model_name'].unique()

for model_name in model_names:
    print(f"\n--- Model: {model_name} ---")
    model_bias_df = bias_df_percentages[bias_df_percentages['model_name'] == model_name].copy()

    # Top k most positive-biased currencies
    top_k_positive_bias = model_bias_df.sort_values(by='Positive_Relative_Change', ascending=False).head(k)
    print(f"Top {k} most positive-biased currencies for {model_name}:")
    display(top_k_positive_bias[['replacement_string', 'Positive_Relative_Change']].round(2))

    # Top k most negatively-biased currencies
    bottom_k_negative_bias = model_bias_df.sort_values(by='Positive_Relative_Change', ascending=True).head(k)
    print(f"Top {k} most negatively-biased currencies for {model_name}:")
    display(bottom_k_negative_bias[['replacement_string', 'Positive_Relative_Change']].round(2))

## Generate Bias Plots per Model:
For each unique model, generate a horizontal bar plot showing the top 10 most positive-biased and bottom 10 most negatively-biased currency replacement strings based on their Positive relative change.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import pandas as pd

# Ensure output_dir exists
# output_dir = './bias_results'
os.makedirs(os.path.join(output_dir, 'currency_wise_pos_bias'), exist_ok=True)

k = 10

model_names = bias_df_percentages['model_name'].unique()

for model_name in model_names:
    print(f"\nGenerating bias plot for model: {model_name}")

    model_bias_df = bias_df_percentages[bias_df_percentages['model_name'] == model_name].copy()

    # Identify top and bottom k
    top_k_positive_bias = model_bias_df.sort_values(by='Positive_Relative_Change', ascending=False).head(k)
    bottom_k_negative_bias = model_bias_df.sort_values(by='Positive_Relative_Change', ascending=True).head(k)

    # Combine and sort for the plot
    plot_data = pd.concat([top_k_positive_bias, bottom_k_negative_bias])
    plot_data = plot_data.sort_values(by='Positive_Relative_Change', ascending=True).reset_index(drop=True)

    plt.figure(figsize=(16, 12))

    norm = plt.Normalize(plot_data['Positive_Relative_Change'].min(),
                         plot_data['Positive_Relative_Change'].max())
    cmap = plt.cm.RdBu
    colors = cmap(norm(plot_data['Positive_Relative_Change']))

    plt.barh(
        plot_data['replacement_string'],
        plot_data['Positive_Relative_Change'],
        color=colors,
        edgecolor='black'
    )

    # --- DOTTED SEPARATION LINE ---
    # linestyle=':' creates the dotted effect. ':' is dotted, '--' is dashed.
    plt.axhline(y=k - 0.5, color='black', linestyle=':', linewidth=3)

    # Optional: Labels to clarify the zones
    plt.text(plt.xlim()[1], k - 0.2, f' TOP {k} ', va='bottom', ha='right', fontsize=28, fontweight='bold')
    plt.text(plt.xlim()[1], k - 0.8, f' BOTTOM {k} ', va='top', ha='right', fontsize=28, fontweight='bold')

    # Vertical reference line
    plt.axvline(x=0, color='gray', linestyle='-', linewidth=0.8, alpha=0.5)

    clean_model_name = model_name.replace('/', '_').replace('-', '_')

    plt.title(f'{clean_model_name}',
              fontsize=25, fontweight='bold', pad=20)
    plt.xlabel('Positive Relative Change (%)', fontsize=28)
    # plt.ylabel('Currency', fontsize=24)
    plt.xticks(fontsize=28)
    plt.yticks(fontsize=28)
    plt.grid(axis='x', linestyle='--', alpha=0.3)
    plt.tight_layout()

    # Save and Show
    file_name = f'sent_class_curr_level_bias_plot_{clean_model_name}.pdf'
    file_path = os.path.join(output_dir, 'currency_wise_pos_bias', file_name)

    plt.savefig(file_path, format='pdf', bbox_inches='tight')
    # plt.show()
    plt.close()

print(f"\nAll bias plots for each model saved to '{output_dir}'.")